In [ ]:
%matplotlib inline
import os
import copy
import subprocess
import matplotlib.pylab as plt
import numpy as np
from os.path import exists

from lib_mogi import *  # includes key functions

# GEOS 627 Inverse Problems and Parameter Estimation, Carl Tape
# GEOS 657 Microwave Remote Sensing, Franz Meyer
# University of Alaska Fairbanks
#
# Modeling subsurface volcanic sources using InSAR data
#
# contributors: Zhong Lu, Franz Meyer, Carl Tape, Amanda McPherson*

In [ ]:
# Define variables - InSAR
nsample = 1100
nline = 980
posting = 40.0
half_wave = 28.3

In [ ]:
# Download and read in data - InSAR
displacement_map_path = 's3://asf-jupyter-data/E451_20000818_20020719.unw'
displacement_map = os.path.basename(displacement_map_path)
if not exists('./E451_20000818_20020719.unw'):
    !aws --region=us-east-1 --no-sign-request s3 cp $displacement_map_path $displacement_map

with open (displacement_map, 'rb') as f:    
    coh = np.fromfile(f, dtype='>f', count=-1)
    
observed_displacement_map = np.reshape(coh, (nline, nsample))

# Scale the measured and unwrapped InSAR phase into surface displacement in cm units and replace all nans with 0
observed_displacement_map = observed_displacement_map*half_wave/2.0/np.pi
where_are_NaNs = np.isnan(observed_displacement_map)
isokay = np.isreal(observed_displacement_map)
observed_displacement_map[where_are_NaNs] = 0

# Create a mask that removes invalid samples (low coherence) from the displacement map
observed_displacement_map_m = np.ma.masked_where(observed_displacement_map==0, observed_displacement_map)

print('data files is %i x %i' % (observed_displacement_map_m.shape))
print('nline = %i' % nline)
print('nsample = %i' % nsample)

In [ ]:
# without mask
plot_model(observed_displacement_map, nline, nsample, posting, dpi=200)
# with mask
plot_model(observed_displacement_map_m, nline, nsample, posting, dpi=200)

In [ ]:
# function to create simulated displacement data based on Mogi source model parameters
# note: this function calls rngchg_mogi() to calculate the displacement field projected onto LOS
# note: this function requires variables nsample, nline, posting
def mogi2insar(x, y, z, volume):    
    # Setting acquisition parameters
    track = -13.3*np.pi / 180.0
    look  =  23.0*np.pi / 180.0
    plook = np.array([-np.sin(look)*np.cos(track), np.sin(look)*np.sin(track), np.cos(look)]).reshape(3,1)
    
    # Defining easting and northing vectors
    northing     = np.arange(0, (nline)*posting, posting) / 1000
    easting      = np.arange(0, (nsample)*posting, posting) / 1000
    northing_mat = np.tile(northing, (nsample, 1))
    easting_mat  = np.transpose(np.tile(easting, (nline, 1)))
    northing_vec = np.reshape(northing_mat, (nline*nsample, 1))
    easting_vec  = np.reshape(easting_mat, (nline*nsample, 1))
    
    # Handing coordinates and model parameters over to the rngchg_mogi function
    calc_range = rngchg_mogi(y, x, z, volume, northing_vec, easting_vec, plook)
    
    # Reshaping surface displacement data derived via calc_forward_model_mogi()
    surface_displacement = np.reshape(calc_range, (nsample,nline))
    
    # return rotated surface displacement
    return np.transpose(np.fliplr(surface_displacement))

In [ ]:
# try out a forward model using mogi2insar()




In [ ]:
# FIX Z AND dV, SEARCH OVER X AND Y





In [ ]:
# Find the minimum
#inds = np.nonzero(misfit_xy == np.amin(misfit_xy))
#indx = inds[0][0]
#indy = inds[1][0]

#print('Source xs : %.2f km' % (xs[indx]))
#print('Source ys : %.2f km' % (ys[indy]))
#print('misfit : %.2e' % (misfit_xy[indx,indy]))